In [19]:
import requests
import tqdm
import h5py

import os

In [6]:
baseUrl = 'http://www.tng-project.org/api/'
headers = {"api-key":"f96eabbae6f336408a63f875f8cce22f"}

In [14]:
def get(path, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically
    return r

def get_save(path, fname, params=None):
    # make HTTP GET request to path
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

#     if r.headers['content-type'] == 'application/json':
#         return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
#         filename = r.headers['content-disposition'].split("filename=")[1]
        with open(fname, 'wb') as f:
            f.write(r.content)
        return # return the filename string

    return r

In [7]:
sim = get(baseUrl + 'TNG50-1/')

In [62]:
sim.keys()

dict_keys(['name', 'description', 'name_alt', 'boxsize', 'z_start', 'z_final', 'cosmology', 'omega_0', 'omega_L', 'omega_B', 'hubble', 'physics_model', 'has_cooling', 'has_starformation', 'has_winds', 'has_blackholes', 'mass_gas', 'mass_dm', 'softening_dm_comoving', 'softening_stars_comoving', 'softening_blackholes_comoving', 'softening_gas_comoving', 'softening_dm_max_phys', 'softening_stars_max_phys', 'softening_blackholes_max_phys', 'softening_gas_max_phys', 'softening_gas_factor', 'softening_gas_comoving_min', 'num_dm', 'num_tr_mc', 'num_tr_vel', 'longids', 'is_uniform', 'is_zoom', 'is_subbox', 'num_files_snapshot', 'num_files_groupcat', 'num_files_rockstar', 'num_files_lhalotree', 'num_files_sublink', 'num_files_ctrees', 'filesize_lhalotree', 'filesize_sublink', 'filesize_ctrees', 'filesize_ics', 'filesize_simulation', 'has_fof', 'has_subfind', 'has_rockstar', 'has_lhalotree', 'has_sublink', 'has_ctrees', 'permission_required', 'num_snapshots', 'url', 'parent_simulation', 'child_s

In [13]:
num_snapshots = sim['num_snapshots']
print(f'num snapshots = {num_snapshots}')

num_files_snapshot = sim['num_files_snapshot']
print(f'num files snapshots = {num_files_snapshot}')

num snapshots = 100
num files snapshots = 680


# Find the galaxy clusters at z = 0

Find the galaxy clusters ($M_{200}$ $\geq$ 1e13)

In [61]:
sim['files']

{'lhalotree': 'http://www.tng-project.org/api/TNG50-1/files/lhalotree/',
 'sublink': 'http://www.tng-project.org/api/TNG50-1/files/sublink/',
 'offsets': 'http://www.tng-project.org/api/TNG50-1/files/offsets/',
 'ics': 'http://www.tng-project.org/api/TNG50-1/files/ics.hdf5',
 'simulation': 'http://www.tng-project.org/api/TNG50-1/files/simulation.hdf5',
 'stellar_circs': 'http://www.tng-project.org/api/TNG50-1/files/stellar_circs.hdf5',
 'subhalo_matching_to_dark': 'http://www.tng-project.org/api/TNG50-1/files/subhalo_matching_to_dark.hdf5',
 'blackhole_mergers': 'http://www.tng-project.org/api/TNG50-1/files/blackhole_mergers.hdf5',
 'blackhole_details': 'http://www.tng-project.org/api/TNG50-1/files/blackhole_details.hdf5',
 'stellar_assembly': 'http://www.tng-project.org/api/TNG50-1/files/stellar_assembly.hdf5',
 'subbox_subhalo_list_0': 'http://www.tng-project.org/api/TNG50-1/files/subbox_subhalo_list.0.hdf5',
 'subbox_subhalo_list_1': 'http://www.tng-project.org/api/TNG50-1/files/sub

In [36]:
url = 'http://www.tng-project.org/api/TNG50-1/files/groupcat-99/'
query_mass = '?Group=Group_M_TopHat200'

get_save(url + query_mass, './initial/Group_M_TopHat200')
print('initial Group_M_TopHat200 saved')

query_firstsub = '?Group=GroupFirstSub'
get_save(url + query_firstsub, './initial/GroupFirstSub')
print('initial GroupFirstSub saved')


query_nsub = '?Group=GroupNsubs'
get_save(url + query_nsub, './initial/GroupNsubs')
print('initial GroupNsubs saved')

initial Group_M_TopHat200 saved
initial GroupFirstSub saved
initial GroupNsubs saved


In [37]:
with h5py.File('./initial/Group_M_TopHat200', 'r') as f:
    M200 = f['Group']['Group_M_TopHat200'][:] * 1e10 / 0.704

In [39]:
cl_index = np.where(M200 >= 1e13)[0]

In [47]:
with h5py.File('./initial/GroupFirstSub', 'r') as f:
    firstsub = f['Group']['GroupFirstSub'][:][cl_index]
    
with h5py.File('./initial/GroupNsubs', 'r') as f:
    nsub = f['Group']['GroupNsubs'][:][cl_index]

In [48]:
firstsub

array([     0,  63864,  96762, 117250, 143880, 167392, 184931, 198182,
       208811, 220595, 229933, 242788, 253861, 264883, 275545, 282779,
       289385, 294866, 300903, 307485, 313692, 319730, 324123, 329508,
       333424, 338446], dtype=int32)

In [49]:
nsub

array([63864, 32898, 20488, 26630, 23512, 17539, 13251, 10629, 11784,
        9338, 12855, 11073, 11022, 10662,  7234,  6606,  5481,  6037,
        6582,  6207,  6038,  4393,  5385,  3916,  5022,  4000],
      dtype=int32)

In [58]:
if not os.path.exists('./snaps/'):
    
    os.mkdir('./snaps/')
    
for i in tqdm.trange(100):
    
    path_name = f'./snaps/snap{i}'
    
    if not os.path.exists(path_name):
        os.mkdir(path_name)

100%|██████████| 100/100 [00:00<00:00, 1566.43it/s]


In [ ]:
for i in tqdm.trange(100):
    
    url = f'http://www.tng-project.org/api/TNG50-1/files/groupcat-{i}/'
    path_name = f'./snaps/snap{i}/'
    
    query_mass = '?Group=Group_M_TopHat200'
    get_save(url + query_mass, path_name + 'Group_M_TopHat200')

    query_firstsub = '?Group=GroupFirstSub'
    get_save(url + query_firstsub, path_name + 'GroupFirstSub')

    query_nsub = '?Group=GroupNsubs'
    get_save(url + query_nsub, path_name + 'GroupNsubs')


 58%|█████▊    | 58/100 [2:16:53<1:32:19, 131.90s/it]

In [52]:
sl = get(sim['files']['sublink'])

In [54]:
sl

{'files': ['http://www.tng-project.org/api/TNG50-1/files/sublink.0.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.1.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.2.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.3.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.4.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.5.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.6.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.7.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.8.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.9.hdf5',
  'http://www.tng-project.org/api/TNG50-1/files/sublink.10.hdf5']}

In [58]:
mass_min = 10**14 / 1e10 * 0.704
# mass_max = 10**12.1 / 1e10 * 0.704

search_query = '?mass__gt=' + str(mass_min)

In [ ]:
for snap in range(100):
    snap_url = f'http://www.tng-project.org/api/TNG50-1/snapshots/{snap}/subhalos/'

    url = snap_url + search_query
    
    subhalos = get(url)
    
    print(subhalos['count'])

In [ ]:
#find halos whose masses are greater than 14.
# link the halos in the direction of redshift
# check gas, star-forming galxy distribution

#reference: IllustriTNG-50 page